In [1]:
import sys, os
sys.path.append(os.path.abspath('/spark-data'))

In [2]:
from utils.Data_Ingestion.data_ingestion import load_files

In [3]:
from pyspark.sql import SparkSession

In [4]:
#Spark Session Builder that builds the Session in which we work upon

spark = SparkSession.builder \
    .appName("ExceltoDataFrame") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:3.4.1_0.19.0") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.crealytics#spark-excel_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3c361c0f-dac0-46af-93cf-3c26e9303b8f;1.0
	confs: [default]
	found com.crealytics#spark-excel_2.12;3.4.1_0.19.0 in central
	found org.apache.poi#poi;5.2.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.commons#commons-collections4;4.4 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found com.zaxxer#SparseBitSet;1.2 in central
	found org.apache.poi#poi-ooxml;5.2.3 in central
	found org.apache.poi#poi-ooxml-lite;5.2.3 in central
	found org.apache.xmlbeans#xmlbeans;5.1.1 in central
	found com.github.virtuald#curvesapi;1.07 in central
	found com.norbitltd#spoiwo_2.12;2.2.1 in central
	found com.github.tototoshi#scala-csv_2.12;1.3.10 in central
	found com.github.pjfanning#excel-streaming-reader;4.0.5 in central
	found com.gi

In [5]:
dfs = load_files(spark, "/spark-data/Cleaned_Csv_File")

2024-09-03 05:16:04,512 - INFO - Successfully loaded CSV file: /spark-data/Cleaned_Csv_File/df_review_cleaned.csv
2024-09-03 05:16:04,923 - INFO - Successfully loaded CSV file: /spark-data/Cleaned_Csv_File/df_product_cleaned.csv
2024-09-03 05:16:05,250 - INFO - Successfully loaded CSV file: /spark-data/Cleaned_Csv_File/df_order_item_cleaned.csv
2024-09-03 05:16:05,548 - INFO - Successfully loaded CSV file: /spark-data/Cleaned_Csv_File/df_customer_cleaned.csv
2024-09-03 05:16:05,879 - INFO - Successfully loaded CSV file: /spark-data/Cleaned_Csv_File/df_order_cleaned.csv


In [6]:
df_customer = dfs["df_df_customer_cleaned"]
df_product = dfs["df_df_product_cleaned"]
df_order = dfs["df_df_order_cleaned"]
df_review = dfs["df_df_review_cleaned"]
df_order_item =dfs["df_df_order_item_cleaned"]

In [7]:
df_customer.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|   +1 (511) 782-4381|    Fergusonville|         Iowa|   91344|1995-02-28|
|          2|   Jennifer|    Jones|douglasedwards@ex...|   +1 (918) 720-2495|     Jenniferport|   New Jersey|   59371|1951-01-27|
|          3|  Catherine|    Crane|douglaskevin@exam...|+1 (381) 601-2272...|  Jenniferchester|West Virginia|   49812|1966-10-07|
|          4|     Lauren|    Irwin|aguirresteven@exa...|Invalid phone number|      New Melissa|     Michigan|   17279|1936-07-28|
|          5|Christopher|   Nelson|dustin50@example.org|+1 (735) 814-4145...|       Hannah

In [8]:
df_order.show()

+--------+-----------+----------+------------+--------------------+--------------------+
|order_id|customer_id|order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+----------+------------+--------------------+--------------------+
|     131|       4958|2020-05-10|         569|7886 Higgins Moto...|14788 Robert Land...|
|     297|         46|2020-03-11|        1331|PSC 4470, Box 299...|16387 Novak Shoal...|
|     330|       1082|2025-05-03|        4010|758 Sarah Plaza P...|938 Brian Road La...|
|      62|       2934|2024-06-01|        1727|7971 Vicki Key Ap...|465 Kramer Way Ap...|
|     266|       3929|2022-05-30|         680|57853 Chen Way La...|055 Butler Extens...|
|      43|        118|2021-09-26|        4884|9822 Anne Trace A...|1511 Hill Keys Ap...|
|      95|       4222|2023-08-27|        3765|Unit 4050 Box 161...|06811 Sean Crossi...|
|     167|       3800|2022-12-09|        6259|16060 Mason Creek...|196 Joshua Meadow...|
|     300|       1337

In [9]:
from utils.Data_Transformation.transformation import add_age_column, add_clv_column

In [10]:
age_added_customer_df = add_age_column(df_customer, "birth_date")

2024-09-03 05:16:23,532 - INFO - Starting to add 'age' column based on the birth date column.
2024-09-03 05:16:23,567 - INFO - Column 'birth_date' found. Calculating age...
2024-09-03 05:16:23,651 - INFO - Age column added successfully.


In [11]:
age_added_customer_df.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+---+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|age|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+---+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|   +1 (511) 782-4381|    Fergusonville|         Iowa|   91344|1995-02-28| 29|
|          2|   Jennifer|    Jones|douglasedwards@ex...|   +1 (918) 720-2495|     Jenniferport|   New Jersey|   59371|1951-01-27| 73|
|          3|  Catherine|    Crane|douglaskevin@exam...|+1 (381) 601-2272...|  Jenniferchester|West Virginia|   49812|1966-10-07| 57|
|          4|     Lauren|    Irwin|aguirresteven@exa...|Invalid phone number|      New Melissa|     Michigan|   17279|1936-07-28| 88|
|          5|Christopher|   Nelson|dustin50@example.org|+1 (73

In [12]:
df_customer_add_clv = add_clv_column(age_added_customer_df, df_order, "total_amount", "customer_id")

2024-09-03 05:16:27,822 - INFO - Starting to add 'CLV' column to the customers DataFrame.
2024-09-03 05:16:27,840 - INFO - Columns validated. Proceeding with CLV calculation.
2024-09-03 05:16:28,010 - INFO - 'CLV' column added successfully, with null values replaced by 0.


In [13]:
df_customer_add_clv.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+---+----+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|age| CLV|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+---+----+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|   +1 (511) 782-4381|    Fergusonville|         Iowa|   91344|1995-02-28| 29|   0|
|          2|   Jennifer|    Jones|douglasedwards@ex...|   +1 (918) 720-2495|     Jenniferport|   New Jersey|   59371|1951-01-27| 73|8766|
|          3|  Catherine|    Crane|douglaskevin@exam...|+1 (381) 601-2272...|  Jenniferchester|West Virginia|   49812|1966-10-07| 57|   0|
|          4|     Lauren|    Irwin|aguirresteven@exa...|Invalid phone number|      New Melissa|     Michigan|   17279|1936-07-28| 88|   0|
|          5|Christopher|  